In [1]:
import pdfplumber
import re
import openai

# OpenAI API Key here
api_key = 0 #github not letting me upload my code with API key
#openai.api_key = api_key
GPT_MODEL = "gpt-4"

In [2]:
keywords = [
    "high torque", "vibration", "high vibration", "excessive vibration",
    "failure", "equipment failure", "system failure", "damage ",
    "bit damage", "tool damage", "slow penetration rate", "torque and drag",
    "spike in torque", "stuck pipe", "high pressure", "low pressure",
    "kick", "gas kick", "lost circulation", "borehole instability",
    "mse", "drag", "high rpm", "low rpm", "temperature spike",
    "mud loss", "bop issues", "chatter", "whirl", "stick-slip",
    "washout", "twist off", "pump failure", "severe wear"
]

def highlight_keywords(text, keywords):
    for keyword in keywords:
        text = re.sub(f"(?i)({keyword})", r'<mark>\\1</mark>', text)
    return text

def extract_sentences_with_keywords(text, keywords):
    sentences_with_keywords = []
    sentences = re.split(r'(?<=[.!?]) +', text)
    for sentence in sentences:
        for keyword in keywords:
            if re.search(f"(?i){keyword}", sentence):
                sentences_with_keywords.append(sentence)
                break
    return sentences_with_keywords


In [3]:
def gpt_summary(text, model=GPT_MODEL, max_tokens=150):
    response = openai.ChatCompletion.create( # Use ChatCompletion for chat models
        model=model,
        messages=[{"role": "user", "content": f"Summarize the following text: {text}"}],
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.5
    )
    return response.choices[0].message.content.strip() # Extract content from message


In [4]:
def process_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        highlighted_text = highlight_keywords(text, keywords)
        sentences_with_keywords = extract_sentences_with_keywords(text, keywords)
        summary = gpt_summary(text)
        return highlighted_text, sentences_with_keywords, summary


In [6]:
# Replace "your_pdf_file.pdf" with the path to a PDF you'd like to analyze
pdf_path = "02July2021.pdf"
highlighted_text, sentences_with_keywords, summary = process_pdf(pdf_path)

print("Highlighted Text:", highlighted_text)
print("********************************************************************************")
print("Sentences with Keywords:", sentences_with_keywords)
print("********************************************************************************")
print("Summary:", summary)


Highlighted Text: Daily Drilling Report Geothermal Resource Group, Inc.
Well ID: Forge 78B-32 Well Name: 78B-32
Field: UTAHFORGE Sect: 32 Town: 26 Rng: 9W County: Beaver State: UT
Report No: 6 Report For 02-Jul-21
Operator: University of Utah Rig: Frontier Rig 16 Spud Date: 28-Jun-21 Daily Cost / Mud ($): ---
Measured Depth (ft): 3009 Last Casing: 11.750 at 2,990 Wellbore: Original Wellbore AFE No. AFE ($) Actual ($)
Vertical Depth (ft): 3009 Next Casing: 7.000 at 8,500 RKB Elevation (ft): 30.40 --- --- ---
Proposed TD (ft): 9500 Last BOP Test: 29-Jun-21 Job Reference RKB (ft): --- --- ---
Hole Made (ft) / Hrs: 0 / 0.0 Next BOP Test: 19-Jul-21 Working Interest: Totals: --- ---
Average ROP (ft/hr): Well Cost ($): ---
Drilling Days (act./plan): 6/21 Flat Days (act./plan): 0/9 Total Days (act./plan): 6/30 Days On Location: 6
Pers/Hrs: Operator: 3 / 36 Contractor: 14 / 168 Service: 4 / 48 Other: 7 / 84 Total: 28 / 336
Safety Summary: No incidents or events reported. Conducted Crown Check, 